In [1]:
# Supress unnecessary warnings so that presentation looks clean
import warnings
warnings.filterwarnings('ignore')

# Read raw data from the file
import pandas as pd #provides data structures to quickly analyze data
import numpy as np
import time


print ("Reading dataset...")
#Read the train dataset
dataset = pd.read_csv('../input/trainv4.csv')
#Read test dataset
dataset_test = pd.read_csv('../input/testv4.csv')

print ("Length of dataset = " + str(len(dataset)))

## cat117 is cont 2
## cat 118 is High vs. Low
print ("Factorizing categorical variables...")
features = dataset.columns
cats = [feat for feat in features if 'cat' in feat]
## print cats
for feat in cats:
    dataset[feat] = pd.factorize(dataset[feat], sort=True)[0]
    dataset_test[feat] = pd.factorize(dataset_test[feat], sort=True)[0]
print ("Finished loading and factorized data.")

Reading dataset...
Length of dataset = 188318
Factorizing categorical variables...
Finished loading and factorized data.


In [2]:
print ("Preprocessing the Data...")
## Response
shift = 200
response = np.log(dataset['loss'].values) + shift

## Drop the response from our dataset
dataset = dataset.drop(['loss', 'id'], 1)

print "Responses:"
print response

Preprocessing the Data...
Responses:
[ 7.78870066  7.30222685  8.07249545 ...,  8.69326862  7.47469844
  8.50749027]


In [6]:
import math
round(math.sqrt(len(dataset)))

434.0

In [12]:
print("-")*50
print "KNN with 1 Fold Validation"
print("-")*50

import math
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
from sklearn import cross_validation
from sklearn.cross_validation import KFold

print ("Creating Training and Validation sets...")
#get the number of rows and columns

seed = 0

X_train, X_val, Y_train, Y_val = cross_validation.train_test_split(dataset, response, test_size=.1, random_state=seed)
print "Length of Validation Set:" + str(len(Y_val))
print "Length of Train Set:" + str(len(Y_train))
print "Ratio: " + str(len(Y_val) / float(len(Y_train)))

seed = 0
print "Training KNN Model..."
## n_jobs=-1: Use all cores
## n_neighbors = Number of Neighbors
model = KNeighborsRegressor(n_neighbors=round(math.sqrt(len(dataset))),n_jobs=-1)
model.fit(X_train, Y_train)

validations_predictions = np.exp(model.predict(X_val)) - shift
print pd.DataFrame(validations_predictions)
pd.DataFrame(validations_predictions).to_csv("KNN.csv", index=False)
    
MAE = mean_absolute_error(np.exp(Y_val)-shift, np.exp(model.predict(X_val))-shift)
print "MAE: " + str(MAE)

--------------------------------------------------
KNN with 1 Fold Validation
--------------------------------------------------
Creating Training and Validation sets...
Length of Validation Set:18832
Length of Train Set:169486
Ratio: 0.111112422265
Training KNN Model...
                 0
0      1943.477201
1      2174.715670
2      1908.533927
3      2174.024626
4      2218.301691
5      2808.088366
6      2306.648928
7      1908.382807
8      2171.237093
9      2032.194497
10     2060.023509
11     2142.296557
12     2095.546639
13     1781.440945
14     2208.022745
15     2232.395735
16     2093.751629
17     1905.935769
18     1996.944586
19     2456.924036
20     1842.662521
21     2319.779187
22     2252.642405
23     1917.847980
24     1674.752245
25     2241.064336
26     1674.275998
27     2387.349387
28     2240.483379
29     2285.391324
...            ...
18802  1803.008587
18803  1693.948042
18804  1964.311811
18805  1790.653735
18806  2176.865518
18807  2011.935753
18808 

In [14]:
predictions = np.exp(model.predict(dataset_test.drop(['id'],1))) - shift
pd.DataFrame(predictions).to_csv("test_resultsKNN.csv", index=False)